# Notebook Info

From the data tables that we have, we try identifying the features that matter the most for forecasting
failures.

For now the data is pulled from the `xdiag` table and failure is imported from the `failure_info` table.

Database Details:
```
# Data
database = 'oasis-prod'
schema = 'xspoc'
table = 'xdiag'

# Failure
database = 'oasis-prod'
schema = 'analysis'
table = 'failure_info'  
```

Note: The tables especially `xdiag` has data from around 900 wells. Querying the entire table may take time. Can try working on a group of wells or single wells for the analysis.

# Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from library import lib_aws

pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings('ignore')

# Initial Analysis

Just to check the timestamps and how the data is spread out in both the tables

In [3]:
%%time
query_initial = """
SELECT
    distinct("NodeID"),
    min("Date") as min_date,
    max("Date") as max_date
FROM xspoc.xdiag
GROUP BY "NodeID"
ORDER BY "NodeID"
"""

# queryinh the entire failure info
query_failures = """
SELECT 
    "NodeID",
    "Last Oil",
    "Start Date",
    "Finish Date",
    "Job Type",
    "Job Bucket",
    "Primary Symptom",
    "Secondary Symptom"
FROM
    analysis.failure_info
ORDER BY "NodeID";
"""

with lib_aws.PostgresRDS(db='oasis-prod', verbose=1) as engine:
    data_info = pd.read_sql(query_initial, engine, parse_dates=['Date'])
    failures = pd.read_sql(query_failures, engine, parse_dates=['Last Oil', 'Start Date', 'Finish Date'])

Connected to oasis-prod DataBase
Connection Closed
Wall time: 52 s


In [52]:
print('Data info')
display(data_info.head())

print('Failure info')
display(failures.head())

Data info


,NodeID,min_date,max_date
0,Aagvik 1-35H,2019-06-21 15:58:34,2020-07-21 10:18:00
1,Acadia 31-25H,2019-05-27 23:33:12,2020-08-15 16:18:15
2,Acklins 12-18H,2019-05-27 23:52:43,2020-08-15 15:29:25
3,Aerabelle 5502 43-7T,2019-05-27 23:49:54,2020-08-15 02:39:51
4,Ak Strangeland 43-12T,2019-05-28 01:20:59,2020-08-15 12:27:00


Failure info


,NodeID,Last Oil,Start Date,Finish Date,Job Type,Job Bucket,Primary Symptom,Secondary Symptom
0,Aagvik 1-35H,2019-11-27,2019-12-02,2019-12-06,TUBING LEAK,TUBING,Mechanically Induced Damage,Solids in Pump
1,Aagvik 5298 41-35 2TX,2019-05-29,2019-06-04,2019-06-25,GAS LIFT,PUMP,Low Production,Blank
2,Acadia 31-25H,2019-03-30,2019-04-10,2019-04-16,"1-1/4"" PUMP",PUMP,Corrosion,Mechanically Induced Damage
3,Acadia 31-25H,2018-04-11,2018-05-05,2018-05-11,TUBING LEAK,TUBING,Corrosion,Sand
4,Acklins 6092 12-18H,2019-12-24,2020-01-02,2020-01-03,POLISH ROD BREAK,ROD,Mechanically Induced Damage,None


In [53]:
"""
Most of the data we have is after '2019-05-01'
So for now we remove all failures before that from the failures df
This will help us pick wells for analysis which have seen failures
"""

fail_cut = pd.Timestamp('2019-05-01')
failures = failures[failures['Start Date'] >= fail_cut].reset_index(drop=True)
failures.head()

,NodeID,Last Oil,Start Date,Finish Date,Job Type,Job Bucket,Primary Symptom,Secondary Symptom
0,Aagvik 1-35H,2019-11-27,2019-12-02,2019-12-06,TUBING LEAK,TUBING,Mechanically Induced Damage,Solids in Pump
1,Aagvik 5298 41-35 2TX,2019-05-29,2019-06-04,2019-06-25,GAS LIFT,PUMP,Low Production,Blank
2,Acklins 6092 12-18H,2019-12-24,2020-01-02,2020-01-03,POLISH ROD BREAK,ROD,Mechanically Induced Damage,None
3,Aerabelle 5502 43-7T,2018-10-10,2019-08-13,2019-08-15,"3/4"" ROD SECTION",ROD,Mechanically Induced Damage,Dropped (X) Amount of Times
4,Alder 6092 43-8H,2019-12-23,2019-12-27,2020-01-07,"1-1/2"" PUMP",PUMP,Loose Connection,None


In [54]:
data_wells = set(data_info.NodeID)
fail_wells = set(failures.NodeID.unique())

print("Wells with Failure:")
display(data_wells & fail_wells) # wells with failure

# print("Wells without Failure (Atleast in the failure info being used):")
# display(data_wells - fail_wells)

Wells with Failure:


{'Aagvik 1-35H',
 'Aerabelle 5502 43-7T',
 'Amazing Grace Federal 11-2H',
 'Anderson 7-18H',
 'Andre 5501 13-4H',
 'Andre 5501 14-5 3B',
 'Andre Shepherd 5501 21-5 3T',
 'Andre Shepherd 5501 21-5 5T',
 'Andrea 5502 44-7T',
 'Anvers Federal 5602 13-18H',
 'Arnold 16X-12H',
 'Arnstad 3-10H',
 'Autumn Wind State 5601 14-16B',
 'B & Rt 2958 13-25H',
 'Baffin 5601 12-18H',
 'Barenthsen 11-20H',
 'Behan 2-29H',
 'Berkner Federal 5602 43-11H',
 'Berquist 34-27H',
 'Berwick 4-2HE',
 'Betsy Federal 2758 24-29H',
 'Beulah Irene Federal 19-18H',
 'Bobby 5602 43-35H',
 'Bonita 5992 42-22H',
 'Bouvardia Federal 2658 12-12H',
 'Bowie 2958 42-21 1H',
 'Brewer 2759 13-15H',
 'Broderson 13-35H',
 'Broderson 2-27H',
 'Burleson 5502 41-7B',
 'Cade 12-19HA',
 'Carl Federal 2658 43-23H',
 'Carol 12-35H',
 'Carson Federal 2658 13-17H',
 'Ceynar 4-18HB',
 'Ceynar 4X-18H',
 'Charlie 5603 43-19H',
 'Chokecherry 2758 11-10B',
 'Christianson 5404 14-34H',
 'Conry Federal 5992 43-21 1H',
 'Contreras 5502 42-7H',


# Data Import

- Features imported from `xspoc.xdiag`

Following are the Features (Columns) to use for the initial analysis:
```
"NodeID"
"Date",
"PPRL",
"MPRL",
"FluidLoadonPump",
"PumpIntakePressure"
```


## Well Specific

In [55]:
well_name = 'Berwick 4-2HE'  # choose from wells which have failure

query_well = """
SELECT 
    "NodeID",
    "Date",
    "PPRL",
    "MPRL",
    "FluidLoadonPump",
    "PumpIntakePressure"
FROM
    xspoc.xdiag
WHERE "NodeID" = '{}'
ORDER BY "NodeID", "Date";
""".format(well_name)

with lib_aws.PostgresRDS(db='oasis-prod', verbose=1) as engine:
    data_well = pd.read_sql(query_well, engine, parse_dates=['Date'])
 
# Just failures for that well
failure_well = failures[failures.NodeID == well_name]
failure_well.reset_index(inplace=True, drop=True)

display(data_well.head())
print("Failure Info")
display(failure_well)

Connected to oasis-prod DataBase
Connection Closed


,NodeID,Date,PPRL,MPRL,FluidLoadonPump,PumpIntakePressure
0,Berwick 4-2HE,2019-05-28 03:52:36,34693.0,16995.0,11662.0,1389.0
1,Berwick 4-2HE,2019-05-28 04:16:41,36057.0,16277.0,11582.0,1409.0
2,Berwick 4-2HE,2019-05-28 06:15:33,35186.0,16830.0,11798.0,1355.0
3,Berwick 4-2HE,2019-05-28 08:20:47,35264.0,17058.0,11315.0,1537.0
4,Berwick 4-2HE,2019-05-28 08:44:53,36165.0,16295.0,11424.0,1510.0


Failure Info


,NodeID,Last Oil,Start Date,Finish Date,Job Type,Job Bucket,Primary Symptom,Secondary Symptom
0,Berwick 4-2HE,2019-10-31,2019-11-05,2019-11-11,"2"" PUMP",PUMP,Scale,Salt


## Group of Wells

In [56]:
%%time
well_list = [
    'Johnsrud 5198 12-18 10T',
    'Andre 5501 14-5 3B',
    'Berwick 4-2HE',
    'Dixon 5602 44-34H',
    'Emma 13-7H',
    'Forland 28-33H',
    'Susie 15-22H'
]

query_list = """
SELECT
    "NodeID",
    "Date",
    "PPRL",
    "MPRL",
    "FluidLoadonPump",
    "PumpIntakePressure"
FROM xspoc.xdiag
WHERE "NodeID" in {}
ORDER BY "NodeID","Date"
""".format(tuple(well_list))

with lib_aws.PostgresRDS(db='oasis-prod') as engine:
    data_list = pd.read_sql(query_list, engine, parse_dates=['Date'])

failure_list = failures[failures.NodeID.isin(well_list)]
failure_list.reset_index(inplace=True, drop=True)

display(data_list.head())
print("Failure info in these in these wells")
display(failure_list)

,NodeID,Date,PPRL,MPRL,FluidLoadonPump,PumpIntakePressure
0,Andre 5501 14-5 3B,2019-05-28 00:32:02,33010.0,15386.0,8997.0,549.0
1,Andre 5501 14-5 3B,2019-05-28 02:11:23,30272.0,17090.0,8643.0,978.0
2,Andre 5501 14-5 3B,2019-05-28 04:20:24,33434.0,15386.0,9606.0,578.0
3,Andre 5501 14-5 3B,2019-05-28 07:07:08,33168.0,16017.0,9745.0,521.0
4,Andre 5501 14-5 3B,2019-05-28 08:49:42,33046.0,15429.0,9069.0,802.0


Failure info in these in these wells


,NodeID,Last Oil,Start Date,Finish Date,Job Type,Job Bucket,Primary Symptom,Secondary Symptom
0,Andre 5501 14-5 3B,2020-03-06,2020-03-10,2020-03-13,"1-3/4"" PUMP",PUMP,Corrosion,Abrasion - Foreign Debris
1,Berwick 4-2HE,2019-10-31,2019-11-05,2019-11-11,"2"" PUMP",PUMP,Scale,Salt
2,Dixon 5602 44-34H,2019-09-06,2019-09-19,2019-09-19,POLISH ROD BREAK,ROD,None,Unknown
3,Emma 13-7H,2020-04-21,2020-06-09,2020-06-12,"1-1/2"" PUMP",PUMP,Abrasion - Foreign Debris,Sand
4,Forland 28-33H,2019-07-15,2019-07-19,2019-07-24,"1-3/4"" PUMP",PUMP,Mechanically Induced Damage,Compression
5,Johnsrud 5198 12-18 10T,2020-03-03,2020-03-10,2020-03-11,TUBING LEAK,TUBING,Compression,Corrosion
6,Johnsrud 5198 12-18 10T,2019-07-18,2019-07-25,2019-08-01,"2"" PUMP",PUMP,Abrasion - Foreign Debris,Mechanically Induced Damage
7,Susie 15-22H,2020-02-15,2020-02-19,2020-02-25,"1"" ROD SECTION",ROD,Fluid Pound,1st Thread Pin


Wall time: 9.2 s


## Entire Feature Data

Running the next query will import the entire dataset from `xspoc.xdiag`. It has around 3,228,303 rows and took around 14min to run the query

In [32]:
# Dont Run This cell for now
%%time
# Querying the features
query_full = """
SELECT 
    "NodeID",
    "Date",
    "PPRL",
    "MPRL",
    "FluidLoadonPump",
    "PumpIntakePressure"
FROM
    xspoc.xdiag
ORDER BY "NodeID", "Date";
"""



with lib_aws.PostgresRDS(db='oasis-prod') as engine:
    data_full = pd.read_sql(query_full, engine, parse_dates=['Date'])
    
data_full.head()

Wall time: 14min 9s


,NodeID,Date,PPRL,MPRL,FluidLoadonPump,PumpIntakePressure
0,Aagvik 1-35H,2019-06-21 15:58:34,27639.0,16811.0,3280.0,NaN
1,Aagvik 1-35H,2019-06-21 16:25:36,27457.0,16752.0,3241.0,NaN
2,Aagvik 1-35H,2019-06-21 18:25:16,27448.0,16594.0,3330.0,NaN
3,Aagvik 1-35H,2019-06-21 18:28:10,27424.0,16595.0,3327.0,NaN
4,Aagvik 1-35H,2019-06-21 20:25:01,27662.0,16711.0,3341.0,NaN


## Combining

Note: the original failure info can be used. However to make it efficient we only use those wells which are present in the feature dataframe (data_well, data_list, data_full)

In [57]:
"""
Before analysing the data we need to merge the information
Transfering info from failures to data (copy of features)
Using a for loop -- may not be very efficient
"""

def fill_null(df, chk_col='PPRL', well_col='NodeID', time_col='Date'):
    """
    This function will fill in Null Values on those dates where no datapoints are present
    Helps Show failures where no data was present
    Will have to take this into account when running analysis 
    """
    data = df.copy()
    # Set time col as index if it is not
    if time_col in data.columns:
        data.set_index(time_col, inplace=True)
    
    data_gp = data.groupby(well_col).resample('1D').max()  # Groupby wellname and resample to Day freq
    data_gp.drop(columns=[well_col], inplace=True)  # Drop these columns as they are present in the index
    data_gp.reset_index(inplace=True)  # Get Back WellCol from
    data_null = data_gp[data_gp.loc[:, chk_col].isnull()]  # Get all null values, which need to be added to the main data file
    data_null.reset_index(inplace=True, drop=True)
    data.reset_index(inplace=True)  # get timestamp back in the column for concating
    data_full = pd.concat([data, data_null], axis=0, ignore_index=True)  # concat null and og files
    data_full.sort_values(by=[well_col, time_col], inplace=True)
    data_full.drop_duplicates(subset=[well_col, time_col], inplace=True)
    data_full.reset_index(drop=True, inplace=True)
    
    return data_full

def failure_merge(df, failure_df, transfer_cols):
    """
    Merges the failures info
    :param df: dataframe to which info is being transferred to. (Should have columns "NodeID" and "Date")
    :param failure_df: Failure info data (Should have columns "NodeID", "Start Date" and "End Data")
    :param cols: Columns which need to be transferred
    """
    merged = df.copy()  
    for col in transfer_cols:
        merged[col] = 'Normal'  # for now putting everything as normal (even NAN's)
        
    for i in failure_df.index:
        well = failure_df.loc[i, 'NodeID']
        t_start = failure_df.loc[i, 'Start Date']
        t_end = failure_df.loc[i, 'Finish Date'] + pd.Timedelta('1 day')  # As we have day based frequency (the times in a day are considered as 00:00:00)
        bool_ = (merged.NodeID == well) & (merged.Date >= t_start) & (merged.Date <= t_end)  # Boolean mask for main data
        merged.loc[bool_, transfer_cols] = failure_df.loc[i, transfer_cols].values
        
    return merged

In [61]:
# Using the list of wells as the data (data_list and failure_list)
# We could choose whichever dataset we wanted

fill_data = fill_null(data_list)  # FIlling in Nan's where data was missing

transfer_col = ['Job Type', 'Job Bucket', 'Primary Symptom', 'Secondary Symptom']
data = failure_merge(fill_data, failure_list, transfer_col)

data.head()

,Date,NodeID,PPRL,MPRL,FluidLoadonPump,PumpIntakePressure,Job Type,Job Bucket,Primary Symptom,Secondary Symptom
0,2019-05-28 00:32:02,Andre 5501 14-5 3B,33010.0,15386.0,8997.0,549.0,Normal,Normal,Normal,Normal
1,2019-05-28 02:11:23,Andre 5501 14-5 3B,30272.0,17090.0,8643.0,978.0,Normal,Normal,Normal,Normal
2,2019-05-28 04:20:24,Andre 5501 14-5 3B,33434.0,15386.0,9606.0,578.0,Normal,Normal,Normal,Normal
3,2019-05-28 07:07:08,Andre 5501 14-5 3B,33168.0,16017.0,9745.0,521.0,Normal,Normal,Normal,Normal
4,2019-05-28 08:49:42,Andre 5501 14-5 3B,33046.0,15429.0,9069.0,802.0,Normal,Normal,Normal,Normal


# Analysis

In [60]:
# Check this out to see if our data looks good for analysis
data.groupby('NodeID').agg({
    'Date': [min, max, 'count'],
    'Job Bucket': ['nunique']
})

Date                           Job Type
                                        min                 max count  nunique
NodeID                                                                        
Andre 5501 14-5 3B      2019-05-28 00:32:02 2020-08-15 15:33:11  2984        2
Berwick 4-2HE           2019-05-28 03:52:36 2020-08-15 15:41:18  3491        2
Dixon 5602 44-34H       2019-05-28 00:34:22 2020-08-15 16:32:22  4167        2
Emma 13-7H              2019-05-30 12:51:37 2020-08-15 15:24:15  3727        2
Forland 28-33H          2019-05-27 23:50:13 2020-08-15 16:10:24  3503        2
Johnsrud 5198 12-18 10T 2019-05-28 07:29:58 2020-08-15 10:49:01  2900        3
Susie 15-22H            2019-05-28 00:16:22 2020-08-15 16:01:57  5270        2